# Salary estimator from listings

The city_state.json file was modified from this github repo [agalea91 - city_to_state_dictionary](https://github.com/agalea91/city_to_state_dictionary/blob/master/city_to_state.py).

The state_abbr.json file was modified from this github repo [JeffPaine - us_state_abbreviations.py](https://gist.github.com/JeffPaine/3083347).

The job posting dataset can be found on Kaggle [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings)

## Setup

In [1]:
%%capture
%pip install pandas xgboost scikit-learn plotly gensim #swifter
print('')

First we must import our packages to manage the dataset. Then we can import the data.

## Setup
Import the many packages

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%aimport JobPostingManager
%aimport Job2Vec
%aimport settings
%autoreload 1

JPM = JobPostingManager.JobPostingManager
J2V = Job2Vec.Job2Vec

Extract the job posting data from the CSV and clean it.

In [3]:
jpm = JPM()
df = jpm.postings.copy()

Average vacation days: 13.445833333333333
Retrieving an existing dataset at c:\dev\job-estimator/archive/clean_postings.bin


In [4]:

from IPython.display import HTML, display
def print_pay_summary(x: pd.DataFrame):
    pay_df = x[pay_cols]
    summary_df = pd.DataFrame([], columns=['pay_col','min','Q1','median','Q3','max','mean','mode'])
    for c in pay_cols:
        qs = [c]+pay_df[c].dropna().quantile([0,.25, .5, .75, 1]).tolist()+[pay_df[c].mean(), pay_df[c].mode().tolist()]
        summary_df.loc[-1] = qs
        summary_df.index+=1
    return summary_df

pay_period_types = ['YEARLY','MONTHLY', 'BIWEEKLY', 'WEEKLY', 'HOURLY']
    
pay_cols = ['max_salary','med_salary','min_salary']
for pay_period in pay_period_types:
    pay_period_df = df.loc[df['pay_period']==pay_period, pay_cols]
    print(pay_period)
    summary = print_pay_summary(pay_period_df)
    
    display(HTML(summary.style.format(precision=0,thousands=",").to_html()))    


YEARLY


,pay_col,min,Q1,median,Q3,max,mean,mode
2,max_salary,"12,000","85,000","120,000","170,000","1,500,000","139,481",[150000.0]
1,med_salary,"10,000","54,000","72,000","105,400","300,500","84,890",[60000.0]
0,min_salary,"10,000","65,000","89,250","120,000","750,000","98,622",[100000.0]


MONTHLY


,pay_col,min,Q1,median,Q3,max,mean,mode
2,max_salary,"12,000","66,996","89,364","119,184","1,320,000","103,996",[85368.0]
1,med_salary,"12,000","25,398","28,716","36,000","300,000","36,286",[30000.0]
0,min_salary,"12,000","45,864","68,208","81,120","1,080,000","73,636",[68208.0]


BIWEEKLY


,pay_col,min,Q1,median,Q3,max,mean,mode
2,max_salary,"53,482","66,976","81,434","87,516","89,965","74,550",[66976.0]
1,med_salary,nan,nan,nan,nan,nan,nan,[]
0,min_salary,"53,482","54,009","58,370","64,519","71,323","60,370",[54009.0]


WEEKLY


,pay_col,min,Q1,median,Q3,max,mean,mode
2,max_salary,"50,804","100,360","109,148","119,658","210,548","111,668","[95726.8, 114884.64000000001, 117303.16, 120931.2]"
1,med_salary,nan,nan,nan,nan,nan,nan,[]
0,min_salary,"50,804","99,684","109,148","119,658","210,548","111,452","[95726.8, 114884.64000000001, 117303.16, 120931.2]"


HOURLY


,pay_col,min,Q1,median,Q3,max,mean,mode
2,max_salary,"10,335","42,679","58,354","96,997","581,985","74,286",[48498.73990982143]
1,med_salary,"10,335","31,621","38,838","54,319","288,994","49,051",[38798.99192785715]
0,min_salary,"10,100","34,919","46,559","77,598","484,987","59,137",[38798.99192785715]


In [5]:
df = jpm.postings_with_pay[['state','avg_salary']].copy()

groups = df.groupby('state')
group_count = groups.count()
df = groups.mean()
df['count'] = group_count
df = df.dropna(axis=1).sort_values(by='avg_salary')

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x = df.index.values, 
        y=df['avg_salary'],
        name="Average Salary",
    ), 
    secondary_y=False)

fig.add_trace(
    go.Scatter(
        x = df.index.values,
        y = df['count'],
        name="Sample Size"
    ),
    secondary_y=True
)

fig.update_xaxes(title_text="State",tickangle=90)

# Set y-axes titles
fig.update_yaxes(title_text="Dollars per year", secondary_y=False)
fig.update_yaxes(title_text="Job Listings (log)", secondary_y=True, type="log")

fig.show()

Dropping rows where every pay column is empty.


In [6]:


#tokenized_df = job2vec.dataset
#print(tokenized_df.head())


In [12]:
import json

from gensim.models import KeyedVectors
job2vec = J2V(jpm.postings)
j2v = job2vec.get_model()
wv = j2v.wv
categories = json.load(open(settings.REPO_PATH + '/assets/job_categories.json'))
vectors = [wv.get_mean_vector(job2vec.tokenize(category)) for category in categories]
#df.to_pickle(settings.REPO_PATH + '/assets/vectorized_categories.bin')

kv = KeyedVectors(vector_size=300)
kv.add_vectors(categories, vectors)


# inspired by https://github.com/piskvorky/gensim/blob/develop/gensim/models/keyedvectors.py#L1165
def get_similar_categories(sentence: str, topn=3) -> list[str, float]:
    tkn = job2vec.tokenize(sentence)
    if len(tkn) == 0:
        return []
    vec = wv.get_mean_vector(tkn)
    categories = kv.similar_by_vector(vec, topn)
    return categories

def categorize(sentence)-> tuple:
    similar = get_similar_categories(sentence, topn=1)
    if not similar:
        return pd.NA
    return similar[0]

# # inspired by https://github.com/piskvorky/gensim/blob/develop/gensim/models/keyedvectors.py#L655
# best = matutils.argsort(distances(), topn=topn + len(all_keys), reverse=True)
top_postings: pd.DataFrame = jpm.postings.head(100)
print(top_postings['title'])
categorized = []

for post in top_postings['title']:
    if not post:
        continue
    category = categorize(post)
    categorized.append([post,str(category)])

categorized = pd.DataFrame(categorized)
print(categorized)

Retrieving an existing model from c:\dev\job-estimator/assets/w2v/w2v.model
0                                 Marketing Coordinator
1                     Mental Health Therapist/Counselor
2                           Assitant Restaurant Manager
3     Senior Elder Law / Trusts and Estates Associat...
4                                    Service Technician
                            ...                        
95                                  Service Coordinator
96                                Fundraising Associate
97                             Social Media Coordinator
98              Professional Window Cleaning Technician
99                                Marketing Coordinator
Name: title, Length: 100, dtype: object
                                                    0  \
0                               Marketing Coordinator   
1                   Mental Health Therapist/Counselor   
2                         Assitant Restaurant Manager   
3   Senior Elder Law / Trusts and Estate

In [ ]:
print(j2v)

Word2Vec<vocab=74766, vector_size=300, alpha=0.025>


In [ ]:
import os
import pickle
from sklearn.cluster import MiniBatchKMeans

kmeans_path = settings.REPO_PATH +'/assets/kmeans/model.bin'   
print('Training....')
model = MiniBatchKMeans(n_clusters=200, max_iter=1000, random_state=1, batch_size=(257 * os.cpu_count()), max_no_improvement=100, verbose=1).fit(X=j2v.wv.vectors)

print('Dumping the model...')
pickle.dump(model, open(kmeans_path,'wb'))

categories = model.cluster_centers_
for i in range(10):
    vectors = j2v.wv.similar_by_vector(model.cluster_centers_[i], topn=10, restrict_vocab=None)
    print(vectors)

Training....
Init 1/1 with method k-means++
Inertia for init 1/1: 576209.375
[MiniBatchKMeans] Reassigning 183 cluster centers.
Minibatch step 1/14545: mean batch inertia: 34.74737354085603
[MiniBatchKMeans] Reassigning 71 cluster centers.
Minibatch step 2/14545: mean batch inertia: 36.48498905642023, ewa inertia: 36.48498905642023
[MiniBatchKMeans] Reassigning 39 cluster centers.
Minibatch step 3/14545: mean batch inertia: 34.97137645914397, ewa inertia: 36.27687668732692
[MiniBatchKMeans] Reassigning 78 cluster centers.
Minibatch step 4/14545: mean batch inertia: 34.03548151750973, ewa inertia: 35.96869871648791
[MiniBatchKMeans] Reassigning 81 cluster centers.
Minibatch step 5/14545: mean batch inertia: 34.1098765807393, ewa inertia: 35.71312217128085
[MiniBatchKMeans] Reassigning 88 cluster centers.
Minibatch step 6/14545: mean batch inertia: 37.8814810311284, ewa inertia: 36.011258101293194
[MiniBatchKMeans] Reassigning 90 cluster centers.
Minibatch step 7/14545: mean batch inerti

In [ ]:
df = jpm.clean[['state','title','job_id']].copy()

df = df.groupby(['state','title']).size().to_frame(name = 'count').reset_index().dropna(axis=1)

fig = go.Figure(data=[go.Scatter3d(x=df['state'], y=df['title'], z=df['count'], mode='markers')])

fig.update_xaxes(title_text="State")
fig.update_yaxes(title_text="Position")

fig.show()

AttributeError: 'JobPostingManager' object has no attribute 'clean'